<a href="https://colab.research.google.com/github/silcheon/UROP_project/blob/master/squat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [51]:
#!ls
#%cd /content/drive
#%cd MyDrive
#%cd drive/MyDrive/UROP2/UROP_project

In [52]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [53]:
files = utils.files_in_order('poses_compressed/squat')

X_train_names, X_test_names = train_test_split(files, test_size=0.3, random_state=39)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)


pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

array(['squat_good_12.npy', 'squat_good_15.npy', 'squat_good_13.npy',
       'squat_good_14.npy', 'squat_good_5.npy', 'squat_good_8.npy',
       'squat_good_6.npy', 'squat_bad_4.npy', 'squat_good_16.npy',
       'squat_good_17.npy', 'squat_good_11.npy'], dtype='<U17')
array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])
array(['squat_bad_7.npy', 'squat_bad_8.npy', 'squat_good_7.npy',
       'squat_good_10.npy', 'squat_good_9.npy'], dtype='<U17')
array([0, 0, 1, 1, 1])


In [54]:
def load_features(names, flag):
    output1 = [] # List of upper arm torso angles
    output2 = [] # List of upper_leg_under_leg_angle
    output3 = [] # List of under_leg_ground_angle
    
    for filename in names:
        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/squat/'+filename)
        else: # 1 flag 이면 새로 만든 값
            ps = load_ps('numpy/'+filename)
        
        poses = ps.poses

        right_present = [1 for pose in poses 
                if pose.rankle.exists and pose.rknee.exists and pose.rhip.exists]
        left_present = [1 for pose in poses
                if pose.lankle.exists and pose.lknee.exists and pose.lhip.exists]
        right_count = sum(right_present)
        left_count = sum(left_present)
        side = 'right' if right_count > left_count else 'left'

        if side == 'right':
            joints = [(pose.rankle, pose.rknee, pose.rhip, pose.neck) for pose in poses]
        else:
            joints = [(pose.lankle, pose.lknee, pose.lhip,pose.neck) for pose in poses]

        # filter out data points where a part does not exist
        joints = [joint for joint in joints if all(part.exists for part in joint)]
        
        torso_vecs = np.array([(joint[3].x - joint[2].x, joint[3].y - joint[2].y) for joint in joints])#몸통벡터
        upper_leg_vecs = np.array([(joint[2].x - joint[1].x, joint[2].y - joint[1].y) for joint in joints])#허벅지 벡터
        under_leg_vecs = np.array([(joint[1].x - joint[0].x, joint[1].y - joint[0].y) for joint in joints])#정강이 벡터
        ground_vecs=np.array([((joint[0].x)+4,0) for joint in joints])#x축과 평행한 벡터(땅벡터)

        torso_vecs = torso_vecs / np.expand_dims(np.linalg.norm(torso_vecs, axis=1), axis=1)
        upper_leg_vecs = upper_leg_vecs / np.expand_dims(np.linalg.norm(upper_leg_vecs, axis=1), axis=1)
        under_leg_vecs = under_leg_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)
        ground_vecs = ground_vecs / np.expand_dims(np.linalg.norm(under_leg_vecs, axis=1), axis=1)

        upper_leg_torso_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, torso_vecs), axis=1), -1.0, 1.0)))
        upper_arm_torso_angle_filtered = medfilt(medfilt(upper_leg_torso_angle, 5), 5)
        
        upper_leg_under_leg_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(upper_leg_vecs, under_leg_vecs), axis=1), -1.0, 1.0)))
        upper_leg_under_leg_angle_filtered = medfilt(medfilt(upper_leg_under_leg_angle, 5), 5)

        under_leg_ground_angle = np.degrees(np.arccos(np.clip(np.sum(np.multiply(under_leg_vecs, ground_vecs), axis=1), -1.0, 1.0)))
        under_leg_ground_angle_filtered = medfilt(medfilt(under_leg_ground_angle, 5), 5)

        output1.append(upper_arm_torso_angle_filtered.tolist())
        output2.append(upper_leg_under_leg_angle_filtered.tolist())
        output3.append(under_leg_ground_angle_filtered.tolist())
     
        
    return output1, output2, output3

X_train_1, X_train_2, X_train_3 = load_features(X_train_names, 0)
X_test_1, X_test_2, X_test_3 = load_features(X_test_names, 0)
print(X_train_names)
print(len(X_train_1))


'''
X_train_1=load_features(X_train_names,0)
X_train_2=load_features(X_train_names,0)
X_test_1=load_features(X_test_names, 0)
X_test_2=load_features(X_test_names, 0)
X_new_names = ['test.npy']
X_new_1= load_features(X_new_names, 1)
X_new_2= load_features(X_new_names, 1)
'''




['squat_good_12.npy' 'squat_good_15.npy' 'squat_good_13.npy'
 'squat_good_14.npy' 'squat_good_5.npy' 'squat_good_8.npy'
 'squat_good_6.npy' 'squat_bad_4.npy' 'squat_good_16.npy'
 'squat_good_17.npy' 'squat_good_11.npy']
11


"\nX_train_1=load_features(X_train_names,0)\nX_train_2=load_features(X_train_names,0)\nX_test_1=load_features(X_test_names, 0)\nX_test_2=load_features(X_test_names, 0)\nX_new_names = ['test.npy']\nX_new_1= load_features(X_new_names, 1)\nX_new_2= load_features(X_new_names, 1)\n"

In [57]:
def KNN(X_names, X_1, X_2, X_3):
    predictions = []
    
    # Store the average distance to good and bad training examples
    f1_good, f1_bad, f2_good, f2_bad, f3_good, f3_bad = [[] for i in range(6)]

    # Compare distance of current test example with all training examples
    for i in range(len(X_train_1)):
        dist1 = utils.DTWDistance(X_train_1[i], X_1)
        dist2 = utils.DTWDistance(X_train_2[i], X_2)
        dist3 = utils.DTWDistance(X_train_3[i], X_3)
            
        if y_train[i]:
            f1_good.append(dist1)
            f2_good.append(dist2)
            f3_good.append(dist3)
                
        else:
            f1_bad.append(dist1)
            f2_bad.append(dist2)
            f3_bad.append(dist3)
                
    good_score = np.mean(f1_good) + np.mean(f2_good) + np.mean(f3_good)
    bad_score  = np.mean(f1_bad) + np.mean(f2_bad) + np.mean(f3_bad)

    print(good_score, bad_score)
    
    #print(good_score, bad_score)
    # dist가 크면 유사도가 적다
    # bad score가 크면 good
    if good_score < bad_score:
        return 1
    else:
        return 0
    
result = []
for i in range(len(X_test_names)) :
    print(X_test_names[i])
    test_label = KNN(X_train_names, X_test_1[i], X_test_2[i], X_test_3[i]) # 기존 test data 
    result.append([X_test_names[i], test_label])
    
#print(X_test_names)
# KNN(X_new_names, X_new_1, X_new_2) # 영상에서 새로 추출한 데이터

squat_bad_7.npy
212.17982255383305 377.8869022441949
squat_bad_8.npy
420.55997545782554 639.8448914765709
squat_good_7.npy
115.39210324534784 510.00954430272424
squat_good_10.npy
125.27214860175084 497.2419786953535
squat_good_9.npy
116.80300096774715 501.83850426496645


In [58]:
print("## TRAIN SET ##")
for i in range(len(X_train_names)) : 
    print(X_train_names[i], end = " ")
    if y_train[i] == 1 :
        print("Good")
    else :
        print("Bad")
        
        
print("\n## TEST RESULT ##")
for r in result : 
    ori_result = r[0].split("_")[1]
    
    print(r[0], end = " ")
    if r[1] == 1 :
        print("Good", end = " ")
    else :
        print("Bad", end = " ")

    if ori_result == 'good' and r[1] == 1 : 
        print("Right")  
    
    elif ori_result == 'bad' and r[1] == 0 : 
        print("Right")
        
    else : print("Wrong")

## TRAIN SET ##
squat_good_12.npy Good
squat_good_15.npy Good
squat_good_13.npy Good
squat_good_14.npy Good
squat_good_5.npy Good
squat_good_8.npy Good
squat_good_6.npy Good
squat_bad_4.npy Bad
squat_good_16.npy Good
squat_good_17.npy Good
squat_good_11.npy Good

## TEST RESULT ##
squat_bad_7.npy Good Wrong
squat_bad_8.npy Good Wrong
squat_good_7.npy Good Right
squat_good_10.npy Good Right
squat_good_9.npy Good Right
